# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 본인의 구글 드라이브에 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Tensorflow Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Data Argumentation & Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Data Augmentation을 적용하세요.(Image Generator)
            * Transfer Learning(VGG16)


## 1.환경설정 

### (1) 데이터셋 폴더 생성
- **세부요구사항**
    - C드라이브에 Datasets라는 폴더를 만드세요.
        - 구글드라이브를 사용하는경우 드라이브 첫 화면에 Datasets 라는 폴더를 만드세요. ('/content/drive/MyDrive/Datasets/')
    - 해당 폴더 안에 Car_Images.zip 파일을 넣으세요.

* 구글 Colab을 이용하는 경우

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/drive/MyDrive/Datasets

mkdir: cannot create directory ‘/content/drive/MyDrive/Datasets’: File exists


### (2) 데이터셋 불러오기 
- **세부요구사항**
    - Car_Images.zip 파일을 C:/Datasets/ 경로에 압축 해제합니다.
    - zipfile 모듈을 이용하거나 다른 방식을 사용해도 됩니다.
        - 참고 자료 : [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 폴더구조(로컬)
        * C:/Datasets/ : 압축파일
        * C:/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 폴더구조(구글드라이브브)
        * /content/drive/MyDrive/Datasets/ : 압축파일
        * /content/drive/MyDrive/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 압축을 해제하면 다음과 같은 두 하위 폴더가 생성됩니다.
        * normal, abnormal : 각 폴더에는 이미지들이 있습니다.
        * 이후 단계에서 해당 경로로 부터 validation, test 셋을 추출하게 됩니다.
        

In [ ]:
import zipfile

In [ ]:
# 압축파일 경로
# 구글 드라이브인 경우 경로에 맞게 지정하세요.
# dataset_path  = '/content/drive/MyDrive/Datasets/'
dataset_path = '/content/drive/MyDrive/Datasets/'

file_path = dataset_path + 'Car_Images.zip'

In [ ]:
!mkdir /content/drive/MyDrive/Datasets/Car_Image_train; 

In [ ]:
# 압축 해제
data = zipfile.ZipFile('/content/drive/MyDrive/Car_Images.zip')
data.extractall('/content/drive/MyDrive/Datasets/Car_Image_train')

### (3) 이미지 저장을 위한 폴더 생성
- **세부요구사항**
    - train, validation, test 을 위해 각각 하위 폴더 normal과 abnormal를 준비합니다.
        - train
            * 정상 이미지 저장소 : C:/Datasets/Car_Images_train/normal/ 
                * 구글드라이브 :   /content/drive/MyDrive/Datasets/Car_Images_train/normal/
            * 파손 이미지 저장소 : C:/Datasets/Car_Images_train/abnormal/
                * 구글드라이브 : /content/drive/MyDrive/Datasets/Car_Images_train/abnormal/
        - val, test 역시 동일한 구조로 생성합니다.
    - 직접 탐색기에서 폴더를 생성할 수도 있고, os 모듈을 이용하여 코드로 작성할 수도 있습니다.
        - 참고 자료 : [os document](https://docs.python.org/3/library/os.html)

In [ ]:
# train 폴더는 압축을 해제하면서 이미 생성 되어 있습니다.

# test 폴더 만들기 os.mkdir()
!mkdir /content/drive/MyDrive/Datasets/Car_Image_test; 
!mkdir /content/drive/MyDrive/Datasets/Car_Image_test/normal
!mkdir /content/drive/MyDrive/Datasets/Car_Image_test/abnormal

# validation 폴더 만들기
!mkdir /content/drive/MyDrive/Datasets/Car_Image_val; 
!mkdir /content/drive/MyDrive/Datasets/Car_Image_val/normal
!mkdir /content/drive/MyDrive/Datasets/Car_Image_val/abnormal


In [147]:
len(os.listdir('/content/drive/MyDrive/Datasets/Car_Image_test/normal/')

SyntaxError: ignored

## 2.데이터 전처리

### (1) 데이터 분할 : Training set | Validation set | Test set 생성
- **세부요구사항**
    - Training set, Validation set, Test set을 만듭니다.
        * size
            * test : 전체에서 20%를 추출합니다.
            * validation : test를 떼어낸 나머지에서 다시 20%를 추출합니다.
        * 데이터는 랜덤하게 추출해야 합니다.
            - random, shutil 모듈을 이용하여 랜덤하게 추출할 수 있습니다.
                - [random document](https://docs.python.org/3/library/random.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
            * 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

#### 1) test, validation 크기를 지정

In [ ]:
import random, shutil, os
import numpy as np

In [ ]:
tr_n_path = '/content/drive/MyDrive/Datasets/Car_Image_train/normal/'
tr_ab_path = '/content/drive/MyDrive/Datasets/Car_Image_train/abnormal/'

In [ ]:
# 전체 이미지 갯수를 확인합니다.
len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path))

(302, 303)

In [ ]:
# test 사이즈 : 전체 이미지의 20%
te_data_num = [round(len(os.listdir(tr_n_path))*0.2), round(len(os.listdir(tr_ab_path))*0.2)]
print(te_data_num)

# validation 사이즈 : test를 제외한 나머지 중에서 20%
val_data_num = [ round((len(os.listdir(tr_n_path))-te_data_num[0])*0.2) , round((len(os.listdir(tr_n_path))-te_data_num[1])*0.2) ]
print(val_data_num)

# train 사이즈
train_data_num = [len(os.listdir(tr_n_path)) - te_data_num[0] - val_data_num[0],
                  len(os.listdir(tr_ab_path))- te_data_num[1] - val_data_num[1]]
print(train_data_num)

[60, 61]
[48, 48]
[194, 194]


In [ ]:
train_n_set = os.listdir(tr_n_path)
train_ab_set = os.listdir(tr_ab_path)

#### 2) test 셋 추출

In [ ]:
random.seed(2023)
te_n_path = '/content/drive/MyDrive/Datasets/Car_Image_test/normal/'
te_ab_path = '/content/drive/MyDrive/Datasets/Car_Image_test/abnormal/'

test_n_set = random.sample(os.listdir(tr_n_path), k = round(len(os.listdir(tr_n_path))*0.2))
test_ab_set = random.sample(os.listdir(tr_ab_path), k = round(len(os.listdir(tr_ab_path))*0.2))
for file in test_n_set :
    now_path = tr_n_path + file
    shutil.move(now_path, te_n_path)

for file in test_ab_set :
    now_path = tr_ab_path + file
    shutil.move(now_path, te_ab_path)

In [ ]:
# 추출 후 이미지 갯수 확인

print(len(os.listdir(te_n_path)), len(os.listdir(te_ab_path)))
print(len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path)))

60 61
242 242


#### 3) validation 셋 추출

In [ ]:
random.seed(2023)
val_n_path = '/content/drive/MyDrive/Datasets/Car_Image_val/normal/'
val_ab_path = '/content/drive/MyDrive/Datasets/Car_Image_val/abnormal/'

val_n_set = random.sample(os.listdir(tr_n_path), k = round(len(os.listdir(tr_n_path))*0.2))
val_ab_set = random.sample(os.listdir(tr_ab_path), k = round(len(os.listdir(tr_ab_path))*0.2))

for file in val_n_set :
    now_path = tr_n_path + file
    shutil.move(now_path, val_n_path)

for file in val_ab_set :
    now_path = tr_ab_path + file
    shutil.move(now_path, val_ab_path)



In [ ]:
# 추출 후 이미지 갯수 확인
len(val_n_set), len(val_ab_set)

print(len(os.listdir(val_n_path)), len(os.listdir(val_ab_path)))
print(len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path)))

48 48
194 194


In [ ]:
print(len(os.listdir(te_n_path)), len(os.listdir(te_ab_path)))
print(len(os.listdir(val_n_path)), len(os.listdir(val_ab_path)))
print(len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path)))

60 61
48 48
194 194


### (2) 데이터 복사 및 이동
- **세부요구사항**
    - 분할된 데이터를 복사 이동합니다.
        - 새로운 폴더에 저장하는 데이터로 "3.모델링I"에서 사용합니다.
        - 기존 폴더는 "4.모델링II > (1) Data Augmentation"에서 사용합니다.
    - Training set | Validation set | Test set의 데이터를 **새로운 폴더**에 복사하세요.
        - 새로운 폴더 명
            * copy_images/trainset
            * copy_images/validset
            * copy_images/testset
        - 새로운 폴더에는 normal, abnormal 파일 모두를 복사합니다. 
            * 파일을 구분하기 위해 abnormal 파일들은 파일명 앞에 접두사 'ab_'를 붙입시다.
        - os, shutil 모듈을 활용하세요.

#### 1) abnormal 파일 복사

* 복사하기 : shutil.copytree()

In [ ]:
shutil.copytree('/content/drive/MyDrive/Datasets/Car_Image_train/abnormal/', dataset_path+'copy_images/trainset/')
shutil.copytree('/content/drive/MyDrive/Datasets/Car_Image_val/abnormal/', dataset_path+'copy_images/validset/')
shutil.copytree('/content/drive/MyDrive/Datasets/Car_Image_test/abnormal/', dataset_path+'copy_images/testset/')

'/content/drive/MyDrive/Datasets/copy_images/testset/'

* abnormal 이미지 이름의 접두어 "ab_" 붙이기 : os.rename

In [ ]:
trainset_path = '/content/drive/MyDrive/Datasets/copy_images/trainset/'
validset_path = '/content/drive/MyDrive/Datasets/copy_images/validset/'
testset_path = '/content/drive/MyDrive/Datasets/copy_images/testset/'

for file in os.listdir(trainset_path) :
    print(file)
    os.rename(trainset_path + file, trainset_path + 'ab_' + file)

for file in os.listdir(validset_path) :
    os.rename(validset_path + file, validset_path + 'ab_' + file)

for file in os.listdir(testset_path) :
    os.rename(testset_path + file, testset_path + 'ab_' + file)



DALLíñE 2023-03-10 18.51.26 - scratched car.png
DALLíñE 2023-03-10 18.51.29 - scratched car.png
DALLíñE 2023-03-10 18.51.32 - scratched car.png
DALLíñE 2023-03-10 18.53.06 - scratched car.png
DALLíñE 2023-03-10 18.53.08 - scratched car.png
DALLíñE 2023-03-10 18.53.58 - slightly damaged car.png
DALLíñE 2023-03-10 18.54.17 - slightly damaged car.png
DALLíñE 2023-03-10 18.54.19 - slightly damaged car.png
DALLíñE 2023-03-10 18.54.24 - slightly damaged car.png
DALLíñE 2023-03-10 22.04.36 - scratched car.png
DALLíñE 2023-03-10 22.04.39 - scratched car.png
DALLíñE 2023-03-10 22.04.42 - scratched car.png
DALLíñE 2023-03-10 22.37.56 - photo of a part of car.png
DALLíñE 2023-03-10 23.28.30 - photo of a part of car without blemish.png
DALLíñE 2023-03-10 23.34.52 - photo of a part of car without blemish.png
DALLíñE 2023-03-10 23.39.41 - photo of a part of car without blemish.png
DALLíñE 2023-03-10 23.55.00 - a part of car without blemish.png
DALLíñE 2023-03-10 2

#### 2) normal 파일 복사

In [ ]:
nomal_train_path = '/content/drive/MyDrive/Datasets/Car_Image_train/normal/'
nomal_val_path = '/content/drive/MyDrive/Datasets/Car_Image_val/normal/'
nomal_test_path = '/content/drive/MyDrive/Datasets/Car_Image_test/normal/'

for file in os.listdir(nomal_train_path) : 
    shutil.copyfile(nomal_train_path+file , dataset_path+'copy_images/trainset/'+file)

for file in os.listdir(nomal_val_path) : 
    shutil.copyfile(nomal_val_path+file , dataset_path+'copy_images/validset/'+file)

for file in os.listdir(nomal_test_path) : 
    shutil.copyfile(nomal_test_path+file , dataset_path+'copy_images/testset/'+file)

* 데이터 갯수 조회

In [ ]:
print(len(os.listdir(dataset_path+'copy_images/trainset/')))
print(len(os.listdir(dataset_path+'copy_images/validset/')))
print(len(os.listdir(dataset_path+'copy_images/testset/')))

388
96
121


In [ ]:
print(len(os.listdir(tr_n_path)) + len(os.listdir(tr_ab_path)))
print(len(os.listdir(val_n_path)) + len(os.listdir(val_ab_path)))
print(len(os.listdir(te_n_path)) + len(os.listdir(te_ab_path)))

388
96
121


## 3.모델링 I
* **세부요구사항**
    * 모델링을 위한 데이터 구조 만들기
        * x : 이미지를 array로 변환합니다.
        * y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

### (1) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다. 
    * 이미지 파일을 불러와 train, valid, test 각각 array 형태로 변환해 봅시다.
        * 각 폴더로 부터 이미지 목록을 만들고
        * 이미지 한장씩 적절한 크기로 로딩하여 (keras.utils.load_img)
            * 이미지가 너무 크면 학습시간이 많이 걸리고, 메모리 부족현상이 발생될 수 있습니다.
            * 이미지 크기를 280 * 280 * 3 이내의 크기를 설정하여 로딩하시오.
            * array로 변환 (keras.utils.img_to_array, np.expand_dims)
        * 데이터셋에 추가합니다.(데이터셋도 array)

#### 1) 이미지 목록 만들기
* train, validation, test 폴더로 부터 이미지 목록을 생성합니다.

In [ ]:
# 이미지 목록 저장
img_train_list = os.listdir(dataset_path+'copy_images/trainset/')
img_valid_list = os.listdir(dataset_path+'copy_images/validset/')
img_test_list = os.listdir(dataset_path+'copy_images/testset/')

In [ ]:
# 메모리, 처리시간을 위해서 이미지 크기 조정
img_size = 227 ## 사이즈 조정 가능

In [ ]:
from keras.utils import load_img, img_to_array

# x_train
x_train = []
train_path = dataset_path+'copy_images/trainset/'
for file in img_train_list :
    img_path = train_path + file
    img = load_img(img_path, target_size=(img_size, img_size))
    img_array = img_to_array(img)
    x_train.append(img_array)
x_train = np.array(x_train)

# x_valid
x_valid = []
val_path = dataset_path+'copy_images/validset/'
for file in img_valid_list :
    img_path = val_path + file
    img = load_img(img_path, target_size=(img_size, img_size))
    img_array = img_to_array(img)
    x_valid.append(img_array)
x_valid = np.array(x_valid)

# x_test

x_test = []
test_path = dataset_path+'copy_images/testset/'
for file in img_test_list :
    img_path = test_path + file
    img = load_img(img_path, target_size=(img_size, img_size))
    img_array = img_to_array(img)
    x_test.append(img_array)
x_test = np.array(x_test)

In [ ]:
x_train.shape, x_valid.shape, x_test.shape

((388, 227, 227, 3), (96, 227, 227, 3), (121, 227, 227, 3))

#### 2) 이미지들을 배열 데이터셋으로 만들기

In [ ]:
###############################################

### (2) y : 클래스 만들기
- **세부요구사항**
    - Training set / Validation set / Test set의 y를 생성합니다.
        - 각각 normal, abnormal 데이터의 갯수를 다시 확인하고
        - normal을 0, abnormal을 1로 지정합니다.

In [ ]:
# 데이터 갯수 확인
print( len(img_train_list) )
print( len([val for val in img_train_list if val.startswith('ab_')]) )
print('---')
print( len(img_valid_list) )
print( len([val for val in img_valid_list if val.startswith('ab_')]) )
print('---')
print( len(img_test_list) )
print( len([val for val in img_test_list if val.startswith('ab_')]) )

388
194
---
96
48
---
121
61


* y_train, y_valid, y_test 만들기
    * normal, abnormal 데이터의 갯수를 다시 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [ ]:
y_train, y_valid, y_test = [0] * x_train.shape[0], [0] * x_valid.shape[0], [0] * x_test.shape[0]

# y_train
for idx, file in enumerate(img_train_list):
    if file.startswith('ab_') : y_train[idx] = 1
y_train = np.array(y_train)
# y_valid
for idx, file in enumerate(img_valid_list):
    if file.startswith('ab_') : y_valid[idx] = 1
y_valid = np.array(y_valid)
# 
for idx, file in enumerate(img_test_list):
    if file.startswith('ab_') : y_test[idx] = 1
y_test = np.array(y_test)

In [ ]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape

((388, 227, 227, 3),
 (388,),
 (96, 227, 227, 3),
 (96,),
 (121, 227, 227, 3),
 (121,))

In [ ]:
print(y_valid)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Min-Max Scaling

In [ ]:
max_x = x_train.max() 
min_x = x_train.min()

x_train = (x_train - min_x)/(max_x - min_x)
x_valid = (x_valid - min_x)/(max_x - min_x)
x_test = (x_test - min_x)/(max_x - min_x)

In [ ]:
y_train.shape

(388,)

### (3) 모델1 - AlexNet(Flatten())
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

from keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization

from sklearn.metrics import confusion_matrix, classification_report

#### 1) 구조 설계

![알뤡스](https://cdn-images-1.medium.com/max/1600/1*jqKHgwZ8alM3K_JRYO_l4w.png)

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape = (227, 227, 3))
hl = Conv2D( filters = 96, kernel_size = (11, 11), strides = (4, 4), activation = 'relu')(il)
hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (5, 5), padding = 'same', activation = 'relu')(hl)
hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 768, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 768, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Flatten()(hl)

hl = Dense(9216, activation = 'relu')(hl)
hl = Dense(4096, activation = 'relu')(hl)
hl = Dense(4096, activation = 'relu')(hl)

hl = Dropout(0.25)(hl)

ol = Dense(1, activation = 'sigmoid')(hl)

model_alexnet = keras.models.Model(il, ol)


#  4. 모델 컴파일
model_alexnet.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
              optimizer = 'adam')
# 모델 요약
model_alexnet.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 27, 27, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 512)       1229312   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 512)      0     

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 15,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_alexnet.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
model_alexnet.fit(x_train, y_train, callbacks = [es, mcp], epochs = 1000, validation_data = (x_valid, y_valid), verbose = 1, batch_size = 32)

Epoch 1/1000
13/13 [==============================] - ETA: 0s - loss: 26.2202 - accuracy: 0.6160
Epoch 1: val_loss improved from inf to 108.48672, saving model to /content/drive/MyDrive/Datasets/mcp_alexnet.h5
13/13 [==============================] - 58s 4s/step - loss: 26.2202 - accuracy: 0.6160 - val_loss: 108.4867 - val_accuracy: 0.5000
Epoch 2/1000
13/13 [==============================] - ETA: 0s - loss: 1.2327 - accuracy: 0.7165
Epoch 2: val_loss improved from 108.48672 to 13.93604, saving model to /content/drive/MyDrive/Datasets/mcp_alexnet.h5
13/13 [==============================] - 15s 1s/step - loss: 1.2327 - accuracy: 0.7165 - val_loss: 13.9360 - val_accuracy: 0.5000
Epoch 3/1000
13/13 [==============================] - ETA: 0s - loss: 0.9105 - accuracy: 0.7294
Epoch 3: val_loss improved from 13.93604 to 6.03990, saving model to /content/drive/MyDrive/Datasets/mcp_alexnet.h5
13/13 [==============================] - 17s 1s/step - loss: 0.9105 - accuracy: 0.7294 - val_loss: 6.0

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred = model_alexnet.predict(x_test)
y_pred = np.where(y_pred>0.5,1,0)

4/4 [==============================] - 1s 276ms/step


In [ ]:
y_pred[:5]

array([[0.5345403 ],
       [0.15227962],
       [0.9935301 ],
       [0.09741173],
       [0.31241253]], dtype=float32)

In [ ]:
y_test[:5]

array([1, 1, 1, 1, 1])

In [ ]:
y_test.shape, y_pred.shape

((121,), (121, 1))

In [ ]:
y_pred = np.where(y_pred>0.5,1,0)

In [ ]:
confusion_matrix(y_test, y_pred)

array([[54,  6],
       [21, 40]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.90      0.80        60
           1       0.87      0.66      0.75        61

    accuracy                           0.78       121
   macro avg       0.79      0.78      0.77       121
weighted avg       0.80      0.78      0.77       121



In [ ]:
x_train.shape, x_valid.shape, x_test.shape

((388, 227, 227, 3), (96, 227, 227, 3), (121, 227, 227, 3))

### (4) 모델2 - AlexNet(GlobalAvgPool2D())
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

In [ ]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape = (227, 227, 3))
hl = Conv2D( filters = 96, kernel_size = (11, 11), strides = (4, 4), activation = 'relu')(il)
hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (5, 5), padding = 'same', activation = 'relu')(hl)
hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 768, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 768, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = keras.layers.GlobalAvgPool2D()(hl)

hl = Dropout(0.25)(hl)

ol = Dense(1, activation = 'sigmoid')(hl)

model_alexnet_GlobalAvgPool = keras.models.Model(il, ol)


#  4. 모델 컴파일
model_alexnet_GlobalAvgPool.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
              optimizer = 'adam')
# 모델 요약
model_alexnet_GlobalAvgPool.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 27, 27, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 512)       1229312   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 512)      0     

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 15,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_alexnet_GlobalAvgPool.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
model_alexnet_GlobalAvgPool.fit(x_train, y_train, callbacks = [es, mcp], epochs = 1000, validation_data = (x_valid, y_valid), verbose = 1, batch_size = 32)

Epoch 1/1000
12/13 [==========================>...] - ETA: 0s - loss: 1.3622 - accuracy: 0.7344
Epoch 1: val_loss improved from inf to 50.13318, saving model to /content/drive/MyDrive/Datasets/mcp_alexnet_GlobalAvgPool.h5
13/13 [==============================] - 7s 336ms/step - loss: 1.3485 - accuracy: 0.7371 - val_loss: 50.1332 - val_accuracy: 0.5000
Epoch 2/1000
12/13 [==========================>...] - ETA: 0s - loss: 0.4782 - accuracy: 0.8333
Epoch 2: val_loss improved from 50.13318 to 12.69131, saving model to /content/drive/MyDrive/Datasets/mcp_alexnet_GlobalAvgPool.h5
13/13 [==============================] - 2s 126ms/step - loss: 0.4872 - accuracy: 0.8299 - val_loss: 12.6913 - val_accuracy: 0.5000
Epoch 3/1000
12/13 [==========================>...] - ETA: 0s - loss: 0.3784 - accuracy: 0.8464
Epoch 3: val_loss improved from 12.69131 to 2.22830, saving model to /content/drive/MyDrive/Datasets/mcp_alexnet_GlobalAvgPool.h5
13/13 [==============================] - 2s 139ms/step - loss

In [ ]:
y_pred = model_alexnet_GlobalAvgPool.predict(x_test)
y_pred = np.where(y_pred>0.5,1,0)

4/4 [==============================] - 0s 38ms/step


In [ ]:
y_pred[:5]

array([[1],
       [1],
       [1],
       [0],
       [1]])

In [ ]:
confusion_matrix(y_test, y_pred)

array([[41, 19],
       [ 5, 56]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.68      0.77        60
           1       0.75      0.92      0.82        61

    accuracy                           0.80       121
   macro avg       0.82      0.80      0.80       121
weighted avg       0.82      0.80      0.80       121



### (5) 모델3 - VGG-16(D구조)

![VGG-16(D구조)**굵은 텍스트**](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FK990l%2FbtqwDJ7C54R%2F664Ksm6gyTGBR1wK3YPDFk%2Fimg.png)

In [ ]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape = (224, 224, 3))
hl = Conv2D( filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(il)
hl = Conv2D( filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Flatten()(hl)

hl = Dense(4096, activation = 'relu')(hl)
hl = Dense(4096, activation = 'relu')(hl)
hl = Dense(1000, activation = 'relu')(hl)

hl = Dropout(0.3)(hl)

ol = Dense(1, activation = 'sigmoid')(hl)

model_vgg_16_flatten = keras.models.Model(il, ol)


#  4. 모델 컴파일
model_vgg_16_flatten.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
              optimizer = 'adam')
# 모델 요약
model_vgg_16_flatten.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856 

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 15,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_vgg_16_flatten.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
model_vgg_16_flatten.fit(x_train, y_train, callbacks = [es, mcp], epochs = 1000, validation_data = (x_valid, y_valid), verbose = 1, batch_size = 16)

Epoch 1/1000
25/25 [==============================] - ETA: 0s - loss: 4.2534 - accuracy: 0.6546
Epoch 1: val_loss improved from inf to 6.54315, saving model to /content/drive/MyDrive/Datasets/mcp_vgg_16_flatten.h5
25/25 [==============================] - 28s 900ms/step - loss: 4.2534 - accuracy: 0.6546 - val_loss: 6.5431 - val_accuracy: 0.5000
Epoch 2/1000
25/25 [==============================] - ETA: 0s - loss: 0.5587 - accuracy: 0.7577
Epoch 2: val_loss improved from 6.54315 to 2.96899, saving model to /content/drive/MyDrive/Datasets/mcp_vgg_16_flatten.h5
25/25 [==============================] - 14s 579ms/step - loss: 0.5587 - accuracy: 0.7577 - val_loss: 2.9690 - val_accuracy: 0.5208
Epoch 3/1000
25/25 [==============================] - ETA: 0s - loss: 0.5114 - accuracy: 0.7809
Epoch 3: val_loss did not improve from 2.96899
25/25 [==============================] - 6s 249ms/step - loss: 0.5114 - accuracy: 0.7809 - val_loss: 3.9956 - val_accuracy: 0.5000
Epoch 4/1000
25/25 [==========

In [ ]:
y_pred = model_vgg_16.predict(x_test)
y_pred = np.where(y_pred>0.5,1,0)

4/4 [==============================] - 1s 144ms/step


In [ ]:
confusion_matrix(y_test, y_pred)

array([[48, 12],
       [ 8, 53]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.80      0.83        60
           1       0.82      0.87      0.84        61

    accuracy                           0.83       121
   macro avg       0.84      0.83      0.83       121
weighted avg       0.84      0.83      0.83       121



#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape = (224, 224, 3))
hl = Conv2D( filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(il)
hl = Conv2D( filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = keras.layers.GlobalAvgPool2D()(hl)

hl = Dropout(0.3)(hl)

ol = Dense(1, activation = 'sigmoid')(hl)

model_vgg_16 = keras.models.Model(il, ol)


#  4. 모델 컴파일
model_vgg_16.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
              optimizer = 'adam')
# 모델 요약
model_vgg_16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856 

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 15,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_vgg_16.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
model_vgg_16.fit(x_train, y_train, callbacks = [es, mcp], epochs = 1000, validation_data = (x_valid, y_valid), verbose = 1, batch_size = 16)

Epoch 1/1000
25/25 [==============================] - ETA: 0s - loss: 0.6705 - accuracy: 0.7629
Epoch 1: val_loss did not improve from 0.56092
25/25 [==============================] - 15s 242ms/step - loss: 0.6705 - accuracy: 0.7629 - val_loss: 3.1715 - val_accuracy: 0.4792
Epoch 2/1000
25/25 [==============================] - ETA: 0s - loss: 0.4631 - accuracy: 0.7912
Epoch 2: val_loss did not improve from 0.56092
25/25 [==============================] - 6s 228ms/step - loss: 0.4631 - accuracy: 0.7912 - val_loss: 2.0001 - val_accuracy: 0.5104
Epoch 3/1000
25/25 [==============================] - ETA: 0s - loss: 0.4462 - accuracy: 0.7887
Epoch 3: val_loss did not improve from 0.56092
25/25 [==============================] - 6s 229ms/step - loss: 0.4462 - accuracy: 0.7887 - val_loss: 1.6753 - val_accuracy: 0.5000
Epoch 4/1000
25/25 [==============================] - ETA: 0s - loss: 0.4170 - accuracy: 0.8247
Epoch 4: val_loss did not improve from 0.56092
25/25 [===========================

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred = model_vgg_16.predict(x_test)
y_pred = np.where(y_pred>0.5,1,0)

4/4 [==============================] - 1s 133ms/step


In [ ]:
confusion_matrix(y_test, y_pred)

array([[48, 12],
       [ 8, 53]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.80      0.83        60
           1       0.82      0.87      0.84        61

    accuracy                           0.83       121
   macro avg       0.84      0.83      0.83       121
weighted avg       0.84      0.83      0.83       121



### (6) 모델4 - VGG-19(E구조)
![VGG-19](https://wikidocs.net/images/page/164796/vgg_Fig_07.jpeg)

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape = (224, 224, 3))
hl = Conv2D( filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(il)
hl = Conv2D( filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = keras.layers.GlobalAvgPool2D()(hl)

hl = Dropout(0.25)(hl)

ol = Dense(1, activation = 'sigmoid')(hl)

model_vgg_19 = keras.models.Model(il, ol)


#  4. 모델 컴파일
model_vgg_19.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
              optimizer = 'adam')
# 모델 요약
model_vgg_19.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856 

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 15,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_vgg_19.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
model_vgg_19.fit(x_train, y_train, callbacks = [es, mcp], epochs = 1000, validation_data = (x_valid, y_valid), verbose = 1, batch_size = 32)

Epoch 1/1000
13/13 [==============================] - ETA: 0s - loss: 0.5759 - accuracy: 0.7397
Epoch 1: val_loss improved from inf to 17.42811, saving model to /content/drive/MyDrive/Datasets/mcp_vgg_19.h5
13/13 [==============================] - 31s 2s/step - loss: 0.5759 - accuracy: 0.7397 - val_loss: 17.4281 - val_accuracy: 0.4896
Epoch 2/1000
13/13 [==============================] - ETA: 0s - loss: 0.4726 - accuracy: 0.7732
Epoch 2: val_loss improved from 17.42811 to 14.76010, saving model to /content/drive/MyDrive/Datasets/mcp_vgg_19.h5
13/13 [==============================] - 7s 566ms/step - loss: 0.4726 - accuracy: 0.7732 - val_loss: 14.7601 - val_accuracy: 0.5000
Epoch 3/1000
13/13 [==============================] - ETA: 0s - loss: 0.4279 - accuracy: 0.8299
Epoch 3: val_loss improved from 14.76010 to 13.60373, saving model to /content/drive/MyDrive/Datasets/mcp_vgg_19.h5
13/13 [==============================] - 7s 575ms/step - loss: 0.4279 - accuracy: 0.8299 - val_loss: 13.603

In [ ]:
y_pred = model_vgg_19.predict(x_test)
y_pred = np.where(y_pred>0.5,1,0)

4/4 [==============================] - 0s 155ms/step


In [ ]:
confusion_matrix(y_test, y_pred)

array([[47, 13],
       [11, 50]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.78      0.80        60
           1       0.79      0.82      0.81        61

    accuracy                           0.80       121
   macro avg       0.80      0.80      0.80       121
weighted avg       0.80      0.80      0.80       121



In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
keras.backend.clear_session()
model_resnet = keras.Sequential()

model_resnet.add(ResNet50(include_top=True, weights='imagenet' ,input_shape=(224, 224, 3), classes=1))
model_resnet.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
              optimizer = 'adam')
model_resnet.summary()

# 모델 요약
model_resnet.summary()

ValueError: ignored

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 15,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_resnet.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
model_resnet.fit(x_train, y_train, callbacks = [es, mcp], epochs = 1000, validation_data = (x_valid, y_valid), verbose = 1, batch_size = 32)

Epoch 1/1000
13/13 [==============================] - ETA: 0s - loss: 1.1047 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 1.54759, saving model to /content/drive/MyDrive/Datasets/mcp_resnet.h5
13/13 [==============================] - 40s 775ms/step - loss: 1.1047 - accuracy: 0.5000 - val_loss: 1.5476 - val_accuracy: 0.5000
Epoch 2/1000
13/13 [==============================] - ETA: 0s - loss: 0.4709 - accuracy: 0.5000
Epoch 2: val_loss did not improve from 1.54759
13/13 [==============================] - 4s 315ms/step - loss: 0.4709 - accuracy: 0.5000 - val_loss: 5.3505 - val_accuracy: 0.5000
Epoch 3/1000
13/13 [==============================] - ETA: 0s - loss: 0.4397 - accuracy: 0.5000
Epoch 3: val_loss did not improve from 1.54759
13/13 [==============================] - 4s 317ms/step - loss: 0.4397 - accuracy: 0.5000 - val_loss: 4.5695 - val_accuracy: 0.5000
Epoch 4/1000
13/13 [==============================] - ETA: 0s - loss: 0.4713 - accuracy: 0.5000
Epoch 4: val_loss 

KeyboardInterrupt: ignored

In [ ]:
y_pred = model_resnet.predict(x_test)
y_pred = np.where(y_pred>0.5,1,0)
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

4/4 [==============================] - 2s 339ms/step


In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 0, 60],
       [ 0, 61]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.50      1.00      0.67        61

    accuracy                           0.50       121
   macro avg       0.25      0.50      0.34       121
weighted avg       0.25      0.50      0.34       121



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 4.모델링 II
* **세부요구사항**
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Data Augmentation을 통해 데이터를 증가 시킵니다.
            - ImageDataGenerator를 사용합니다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.
            - VGG16(이미지넷)을 사용해 봅시다.

### (1) Data Augmentation
- **세부요구사항**
    * 모델 학습에 이용할 이미지 데이터를 증강시키세요.
    * Keras의 ImageDataGenerator를 이용
        - [ImageDataGenerator document](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

    * image generator를 이용하여 학습
        * 모델 구조는 이미 생성한 1,2,3 중 하나를 선택하여 학습


In [ ]:
!mkdir /content/drive/MyDrive/Datasets/Data_Augmentation

In [ ]:
Data_Augmentation_path = '/content/drive/MyDrive/Datasets/Data_Augmentation/'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = 224 ## 사이즈 조정 가능
dataset_path = '/content/drive/MyDrive/Datasets/'
train_path = dataset_path+'Car_Image_train/'
valid_path = dataset_path+'Car_Image_val/'
test_path = dataset_path+'Car_Image_test/'

#### 1) ImageGenerator 생성
* ImageDataGenerator 함수 사용
    * 주요 옵션
        * rotation_range: 무작위 회전을 적용할 각도 범위
        * zoom_range: 무작위 줌을 적용할 범위 [1-zoom_range, 1+zoom_range]
        * horizontal_flip: 무작위 좌우반전을 적용할지 여부
        * vertical_flip: 무작위 상하반전을 적용할지 여부
        * rescale: 텐서의 모든 값을 rescale 값으로 나누어줌 (이 경우에는 255로 나누어서 0~1사이의 값으로 변경)

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255,
                             rotation_range=30,       # 이미지 회전
                             width_shift_range=0.05,  # 이미지 좌우 이동
                             height_shift_range=0.05, # 이미지 상하 이동
                             zoom_range=0.2,          # 확대/축소 범위
                             shear_range=0.15,        # 비스듬히 늘림
                             horizontal_flip=True,    # 가로 전환
                             vertical_flip=True)      # 세로 전환

testgen = ImageDataGenerator(rescale = 1./255)



:#### 2) 경로로 부터 이미지 불러 올 준비
* .flow_from_directory 이용
    * 디렉토리에서 이미지를 가져와서 데이터 증강을 적용하고 batch 단위로 제공하는 generator를 생성합니다.
    * 이미지를 불러올 때 target_size로 크기를 맞추고, 
    * class_mode로 이진 분류(binary)를 수행하도록 지정합니다.


In [ ]:
train_generator = datagen.flow_from_directory(
    train_path,
    batch_size = 32,
    target_size = (img_size, img_size),
    class_mode = 'binary'
)

valid_generator = testgen.flow_from_directory(
    valid_path,
    batch_size = 32,
    target_size = (img_size, img_size),
    class_mode = 'binary'
)

test_generator = testgen.flow_from_directory(
    test_path,
    batch_size = 32,
    target_size = (img_size, img_size),
    class_mode = 'binary'
)

Found 388 images belonging to 2 classes.
Found 96 images belonging to 2 classes.
Found 121 images belonging to 2 classes.


#### 3) 학습
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 train_generator 이용. 
    - validation_data = valid_generator 지정
    - Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

![알뤡스](https://cdn-images-1.medium.com/max/1600/1*jqKHgwZ8alM3K_JRYO_l4w.png)

* 구조 설계

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

from keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization

from sklearn.metrics import confusion_matrix, classification_report

In [126]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 7,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_alexnet_data_augmentation.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

loss_array = [0] * 5
accuracy_array = [0] * 5

In [127]:
for i in range(5) :
    # 1. 세션 클리어
    keras.backend.clear_session()

    # 2. 모델 사슬처럼 엮기
    il = Input(shape = (227, 227, 3))
    hl = Conv2D( filters = 96, kernel_size = (11, 11), strides = (4, 4), activation = 'relu')(il)
    hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
    hl = BatchNormalization()(hl)

    hl = Conv2D( filters = 512, kernel_size = (5, 5), padding = 'same', activation = 'relu')(hl)
    hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
    hl = BatchNormalization()(hl)

    hl = Conv2D( filters = 768, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
    hl = Conv2D( filters = 768, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
    hl = Conv2D( filters = 512, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
    hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
    hl = BatchNormalization()(hl)

    hl = keras.layers.GlobalAvgPool2D()(hl)

    hl = Dropout(0.3)(hl)

    ol = Dense(1, activation = 'sigmoid')(hl)

    model_alexnet_GlobalAvgPool = keras.models.Model(il, ol)


    #  4. 모델 컴파일
    model_alexnet_GlobalAvgPool.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
                optimizer = 'adam')
    # 모델 요약
    model_alexnet_GlobalAvgPool.summary()

    hist = model_alexnet_GlobalAvgPool.fit(train_generator, epochs=10000, validation_data=valid_generator, verbose = 1, callbacks=[es, mcp], batch_size=32)

    evaluate_result = model_alexnet_GlobalAvgPool.evaluate(test_generator)
    loss_array[i] = evaluate_result[0] # loss
    accuracy_array[i] = evaluate_result[1] # loss

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 27, 27, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 512)       1229312   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 512)      0     

In [131]:
print(loss_array)
print('---------------------------')
print(accuracy_array)
print('---------------------------')
print('loss mean :', np.array(loss_array).mean())
print('accuracy mean :',np.array(accuracy_array).mean())

[0.45521268248558044, 0.3837905526161194, 0.4340907335281372, 0.39785516262054443, 0.4924594461917877]
---------------------------
[0.8347107172012329, 0.8677685856819153, 0.8347107172012329, 0.8429751992225647, 0.8099173307418823]
---------------------------
loss mean : 0.4326817154884338
accuracy mean : 0.8380165100097656


* 학습
    * EarlyStopping 설정하기
    * 학습 데이터에 train_generator, validation_data=valid_generator 사용

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 7,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_alexnet_data_augmentation.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
hist = model_alexnet_GlobalAvgPool.fit(train_generator, epochs=10000, validation_data=valid_generator, verbose = 1, callbacks=[es, mcp], batch_size=32)

Epoch 1/10000
13/13 [==============================] - ETA: 0s - loss: 0.3434 - accuracy: 0.8402
Epoch 1: val_loss improved from inf to 0.35804, saving model to /content/drive/MyDrive/Datasets/mcp_alexnet_data_augmentation.h5
13/13 [==============================] - 22s 2s/step - loss: 0.3434 - accuracy: 0.8402 - val_loss: 0.3580 - val_accuracy: 0.8125
Epoch 2/10000
13/13 [==============================] - ETA: 0s - loss: 0.3422 - accuracy: 0.8686
Epoch 2: val_loss did not improve from 0.35804
13/13 [==============================] - 19s 1s/step - loss: 0.3422 - accuracy: 0.8686 - val_loss: 0.3626 - val_accuracy: 0.8229
Epoch 3/10000
13/13 [==============================] - ETA: 0s - loss: 0.3369 - accuracy: 0.8351
Epoch 3: val_loss did not improve from 0.35804
13/13 [==============================] - 19s 1s/step - loss: 0.3369 - accuracy: 0.8351 - val_loss: 0.3659 - val_accuracy: 0.8125
Epoch 4/10000
13/13 [==============================] - ETA: 0s - loss: 0.3435 - accuracy: 0.8505
Ep

In [132]:
y_pred = model_alexnet_GlobalAvgPool.predict(x_test)

4/4 [==============================] - 0s 38ms/step


In [133]:
y_pred = np.where(y_pred>0.5,0,1)
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.77      0.80        60
           1       0.79      0.85      0.82        61

    accuracy                           0.81       121
   macro avg       0.81      0.81      0.81       121
weighted avg       0.81      0.81      0.81       121



#### 4) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

In [123]:
result = model_alexnet_GlobalAvgPool.evaluate(test_generator)

4/4 [==============================] - 5s 908ms/step - loss: 0.9332 - accuracy: 0.8430


In [124]:
result[0]

0.9332460761070251

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def create_model(verbose = False) :
    input_tensor = Input(shape = (224, 224, 3))
    pretrained_model = Xception(input)


ModuleNotFoundError: ignored

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 15,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_resnet50.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
hist = model_resnet50.fit(train_generator, epochs=10000, validation_data=valid_generator, verbose = 1, callbacks=[es, mcp], batch_size=32)

Epoch 1/10000
13/13 [==============================] - ETA: 0s - loss: 1.4187 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 0.69437, saving model to /content/drive/MyDrive/Datasets/mcp_resnet50.h5
13/13 [==============================] - 54s 2s/step - loss: 1.4187 - accuracy: 0.5000 - val_loss: 0.6944 - val_accuracy: 0.5000
Epoch 2/10000
13/13 [==============================] - ETA: 0s - loss: 0.5793 - accuracy: 0.5000
Epoch 2: val_loss did not improve from 0.69437
13/13 [==============================] - 20s 2s/step - loss: 0.5793 - accuracy: 0.5000 - val_loss: 3.1621 - val_accuracy: 0.5000
Epoch 3/10000
13/13 [==============================] - ETA: 0s - loss: 0.5702 - accuracy: 0.5000
Epoch 3: val_loss did not improve from 0.69437
13/13 [==============================] - 19s 1s/step - loss: 0.5702 - accuracy: 0.5000 - val_loss: 0.7458 - val_accuracy: 0.5000
Epoch 4/10000
13/13 [==============================] - ETA: 0s - loss: 0.5088 - accuracy: 0.5000
Epoch 4: val_loss d

KeyboardInterrupt: ignored

### (2) Transfer Learning
- **세부요구사항**
    * VGG16 모델은 1000개의 클래스를 분류하는 데 사용된 ImageNet 데이터셋을 기반으로 사전 학습된 가중치를 가지고 있습니다. 
        * 따라서 이 모델은 이미지 분류 문제에 대한 높은 성능을 보입니다.
        * 이 모델은 보통 전이학습(transfer learning)에서 기본적으로 사용되며, 특히 대규모 데이터셋이 없을 때는 기본 모델로 사용되어 fine-tuning을 수행합니다.
    * VGG16 함수로 부터 base_model 저장


In [135]:
# The identity block
def identity_block(X, f, filters, stage, block):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    # first step of main path
    X = tf.keras.layers.Conv2D(filters=F1, kernel_size=1, strides=1, padding='valid', name=conv_name_base + '2a',
                              kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    # second step of main path
    X = tf.keras.layers.Conv2D(filters=F2, kernel_size=f, strides=1, padding='same', name=conv_name_base + '2b',
                              kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
    X = tf.keras.layers.Activation('relu')(X) 
    
    # third step of main path
    X = tf.keras.layers.Conv2D(filters=F3, kernel_size=1, strides=1, padding='valid', name=conv_name_base + '2c',
                              kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2c')(X)
    
    # add shortcut value and pass it through a ReLU activation
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    
    return X

In [134]:
# Convolutional Block
def convolutional_block(X, f, filters, stage, block, s=2):
    conv_name_base = 'res'+str(stage)+block+'_branch'
    bn_name_base = 'bn'+str(stage)+block+'_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    # first step of main path
    X = tf.keras.layers.Conv2D(filters=F1, kernel_size=1, strides=s, padding='valid', name=conv_name_base+'2a',
                              kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    # second step of main path
    X = tf.keras.layers.Conv2D(filters=F2, kernel_size=f, strides=1, padding='same', name=conv_name_base+'2b',
                              kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    # third step of main path
    X = tf.keras.layers.Conv2D(filters=F3, kernel_size=1, strides=1, padding='valid', name=conv_name_base+'2c',
                              kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2c')(X)
    
    # shortcut path
    X_shortcut = tf.keras.layers.Conv2D(filters=F3, kernel_size=1, strides=s, padding='valid', name=conv_name_base+'1',
                                       kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'1')(X_shortcut)
    
    # Add and pass it through a ReLU activation
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    
    return X

In [142]:
# ResNet50
def ResNet50(input_shape=(224,224,3), classes=1):
    X_input = tf.keras.layers.Input(input_shape)
    
    # zero padding
    X = tf.keras.layers.ZeroPadding2D((3,3))(X_input)
    
    # stage 1
    X = tf.keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, name='conv1',
                              kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis=3, name='bn_conv1')(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2))(X)
    
    # stage 2
    X = convolutional_block(X, f=3, filters=[64,64,256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64,64,256], stage=2, block='b')
    X = identity_block(X, 3, [64,64,256], stage=2, block='c')
    
    # stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage = 3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage = 3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage = 3, block='d')
    
    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage = 4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage = 4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage = 4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage = 4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage = 4, block='f')
 
    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage = 5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage = 5, block='c')
    
    # AVGPOOL
    X = tf.keras.layers.AveragePooling2D()(X)
    
    # output layer
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(classes, activation='sigmoid', name='fc'+str(classes),
                             kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    
    # Create Model
    model = tf.keras.models.Model(inputs=X_input, outputs=X, name='ResNet50')
    
    return model

In [143]:
model = ResNet50(input_shape = (224, 224, 3), classes = 1)
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_3 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_5[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  9472        ['zero_padding2d_3[0][0]']       
                                )                                                          

In [144]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 5,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_resnet50.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [145]:
hist = model.fit(train_generator, epochs=10000, validation_data=valid_generator, verbose = 1, callbacks=[es, mcp], batch_size=32)

Epoch 1/10000
13/13 [==============================] - ETA: 0s - loss: 2.0325 - acc: 0.7062
Epoch 1: val_loss improved from inf to 13.85926, saving model to /content/drive/MyDrive/Datasets/mcp_resnet50.h5
13/13 [==============================] - 53s 2s/step - loss: 2.0325 - acc: 0.7062 - val_loss: 13.8593 - val_acc: 0.5000
Epoch 2/10000
13/13 [==============================] - ETA: 0s - loss: 2.0864 - acc: 0.6933
Epoch 2: val_loss did not improve from 13.85926
13/13 [==============================] - 20s 2s/step - loss: 2.0864 - acc: 0.6933 - val_loss: 79.6319 - val_acc: 0.5000
Epoch 3/10000
13/13 [==============================] - ETA: 0s - loss: 1.7859 - acc: 0.6856
Epoch 3: val_loss did not improve from 13.85926
13/13 [==============================] - 19s 1s/step - loss: 1.7859 - acc: 0.6856 - val_loss: 155.4107 - val_acc: 0.5000
Epoch 4/10000
13/13 [==============================] - ETA: 0s - loss: 0.7966 - acc: 0.7758
Epoch 4: val_loss did not improve from 13.85926
13/13 [=======

In [ ]:
from tensorflow.keras.applications import VGG16

#### 1) VGG16 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 True로 설정하여 모델의 모든 레이어들이 fine-tuning에 대해 업데이트되도록 합니다.


In [ ]:
base_model = VGG16(                 )




#### 2) VGG16과 연결한 구조 설계
* VGG16을 불러와서 Flatten, Dense 등으로 레이어 연결하기

#### 3) 학습
- **세부요구사항**
    - 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
    - 데이터
        * Image Generator를 연결하거나
        * 기존 train, validation 셋을 이용해도 됩니다.
        - Early Stopping을 반드시 사용하세요.
        - 최적의 가중치를 모델에 적용하세요.

#### 4) 성능 평가